In [ ]:
!pip install langgraph==0.3 

In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict, List

# ---------------------------------------
# 상태 스키마 정의
# ---------------------------------------

class CodeState(TypedDict, total=False):
    code: str
    style_issues: List[str]
    bug_issues: List[str]
    optimize_suggestions: List[str]
    improved_code: str

# ---------------------------------------
# 각 노드 정의
# ---------------------------------------

def style_check_node(state: CodeState) -> CodeState:
    code = state["code"]
    issues = []
    if "return" in code and not code.strip().startswith("def"):
        issues.append("함수가 블록 형태로 작성되어야 합니다.")
    state["style_issues"] = issues
    return state

def bug_check_node(state: CodeState) -> CodeState:
    code = state["code"]
    bugs = []
    if "=" not in code:
        bugs.append("변수 초기화가 누락되었을 수 있습니다.")
    state["bug_issues"] = bugs
    return state

def optimize_check_node(state: CodeState) -> CodeState:
    code = state["code"]
    optimizations = []
    if "for" in code and "range(100000)" in code:
        optimizations.append("루프 최적화를 고려하세요.")
    state["optimize_suggestions"] = optimizations
    return state

def improve_code_node(state: CodeState) -> CodeState:
    original = state["code"]
    improved = original.replace("def ", "def improved_")  # 단순히 함수명 변경
    state["improved_code"] = improved
    return state

# ---------------------------------------
# LangGraph 흐름 구성
# ---------------------------------------

graph = StateGraph(state_schema=CodeState)

graph.add_node("style_check", style_check_node)
graph.add_node("bug_check", bug_check_node)
graph.add_node("optimize_check", optimize_check_node)
graph.add_node("code_improve", improve_code_node)

graph.add_edge("style_check", "bug_check")
graph.add_edge("bug_check", "optimize_check")
graph.add_edge("optimize_check", "code_improve")

graph.set_entry_point("style_check")
graph.set_finish_point("code_improve")

app = graph.compile()

# ---------------------------------------
# 실행 예시
# ---------------------------------------

input_code = {
    "code": "def add(x,y): return x+y"
}

result = app.invoke(input_code)
print("🧪 개선된 코드:")
print(result["improved_code"])